In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau



# Load the data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

Y_train = train["label"]
# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

# free some space
del train 

X_train = X_train / 255.0
test = test / 255.0

X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
Y_train = to_categorical(Y_train, num_classes = 10)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1)



In [24]:
def build_model():
    model = Sequential()

    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu', input_shape = (28,28,1)))
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                     activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))


    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation = "softmax"))
    return model
    
size_ensemble = 4
models = []
for i in range(0,size_ensemble):
    model = build_model()
    model.compile(optimizer = 'Nadam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    models.append(model)

In [ ]:
epochs = 5
batch_size = 128
for model in models:
    model.fit(X_train,Y_train, batch_size,epochs=epochs,validation_data=(X_val,Y_val))

Train on 37800 samples, validate on 4200 samples
Epoch 1/5
37800/37800 [==============================] - 238s - loss: 0.1240 - acc: 0.9617 - val_loss: 0.0734 - val_acc: 0.9762
Epoch 2/5
37800/37800 [==============================] - 236s - loss: 0.0643 - acc: 0.9810 - val_loss: 0.0554 - val_acc: 0.9850
Epoch 3/5
37800/37800 [==============================] - 237s - loss: 0.0518 - acc: 0.9839 - val_loss: 0.0500 - val_acc: 0.9857
Epoch 4/5
37800/37800 [==============================] - 244s - loss: 0.0472 - acc: 0.9862 - val_loss: 0.0900 - val_acc: 0.9738
Epoch 5/5
37800/37800 [==============================] - 243s - loss: 0.0405 - acc: 0.9874 - val_loss: 0.0482 - val_acc: 0.9869
Train on 37800 samples, validate on 4200 samples
Epoch 1/5
37800/37800 [==============================] - 238s - loss: 0.2457 - acc: 0.9224 - val_loss: 0.0541 - val_acc: 0.9826
Epoch 2/5
37800/37800 [==============================] - 232s - loss: 0.0703 - acc: 0.9782 - val_loss: 0.0506 - val_acc: 0.9848
Epoch 

In [ ]:
# predict results
results = None
for model in models:
    if results is None:
        results = model.predict(test)
    else:
        results += model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("cnn_mnist_datagen.csv",index=False)